In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# BigQuery
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project="project='TC-DA-1")




In [3]:
# Construct a reference to the "Adventure work 2005" dataset
dataset_ref = client.dataset("adwentureworks_db", project="tc-da-1")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
def query_to_df(query):
    # transfers query results to pandas dataframe for easy manipulating
    return(client.query(query, project="tc-da-1").result().to_dataframe())

def get_schema(table):
    # retreives the schema as a printed object
    return(client.get_table(table).schema)

In [5]:
# List all the tables in the  dataset
tables = list(client.list_tables(dataset_ref))
print(f"There are {len(tables)} tables in the dataset")
# Print names of all tables in the dataset 
for table in tables:  
    print(table.table_id)


There are 65 tables in the dataset
EmployeeDepartmentHistory
address
addresstype
billofmaterials
contact
contactcreditcard
contacttype
countryregion
countryregioncurrency
creditcard
culture
currency
currencyrate
customer
customeraddress
department
document
employee
employeeaddress
employeedepartmenthistory
employeepayhistory
individual
location
product
productcategory
productcosthistory
productdescription
productdocument
productinventory
productlistpricehistory
productmodel
productmodelillustration
productmodelproductdescriptionculture
productphoto
productproductphoto
productreview
productsubcategory
productvendor
purchaseorderdetail
purchaseorderheader
salesorderdetail
salesorderheader
salesorderheadersalesreason
salesperson
salespersonquotahistory
salesreason
salestaxrate
salesterritory
salesterritoryhistory
scrapreason
shift
shipmethod
shoppingcartitem
specialoffer
specialofferproduct
stateprovince
storecontact
transactionhistory
transactionhistoryarchive
unitmeasure
vendor
vendorad

In [6]:
for table in tables:
    print(f"{table.table_id}: {get_schema(f'tc-da-1.adwentureworks_db.{table.table_id}')}")

EmployeeDepartmentHistory: [SchemaField('EmployeeID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('DepartmentID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('ShiftID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('StartDate', 'DATETIME', 'NULLABLE', None, (), None), SchemaField('EndDate', 'DATETIME', 'NULLABLE', None, (), None), SchemaField('ModifiedDate', 'TIMESTAMP', 'NULLABLE', None, (), None)]
address: [SchemaField('AddressID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('AddressLine1', 'STRING', 'NULLABLE', None, (), None), SchemaField('AddressLine2', 'STRING', 'NULLABLE', None, (), None), SchemaField('City', 'STRING', 'NULLABLE', None, (), None), SchemaField('StateProvinceID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('PostalCode', 'STRING', 'NULLABLE', None, (), None), SchemaField('rowguid', 'BYTES', 'NULLABLE', None, (), None), SchemaField('ModifiedDate', 'TIMESTAMP', 'NULLABLE', None, (), None)]
addresstype: [SchemaField('AddressTypeID

In [7]:
 QUERY = (
     """
SELECT p.ProductId, p.Name, p.ProductNumber, p.size, p.color, p.ProductSubcategoryId, s.Name AS SubcategoryName
FROM `tc-da-1.adwentureworks_db.product`AS p
INNER JOIN `tc-da-1.adwentureworks_db.productsubcategory` AS s 
ON p.ProductSubcategoryId=s.ProductSubcategoryId
ORDER BY s.Name 
     """
 )
    
query_to_df(QUERY)

,ProductId,Name,ProductNumber,size,color,ProductSubcategoryId,SubcategoryName
0,855,"Men's Bib-Shorts, S",SB-M891-S,S,Multi,18,Bib-Shorts
1,857,"Men's Bib-Shorts, L",SB-M891-L,L,Multi,18,Bib-Shorts
2,856,"Men's Bib-Shorts, M",SB-M891-M,M,Multi,18,Bib-Shorts
3,876,Hitch Rack - 4-Bike,RA-H123,None,None,26,Bike Racks
4,879,All-Purpose Bike Stand,ST-1401,None,None,27,Bike Stands
...,...,...,...,...,...,...,...
290,828,HL Road Rear Wheel,RW-R820,None,Black,17,Wheels
291,823,LL Mountain Rear Wheel,RW-M423,None,Black,17,Wheels
292,824,ML Mountain Rear Wheel,RW-M762,None,Black,17,Wheels
293,825,HL Mountain Rear Wheel,RW-M928,None,Black,17,Wheels


In [8]:
QUERY = (
     """
SELECT p.ProductId, p.Name, p.ProductNumber, p.size, p.color, p.ProductSubcategoryId, s.Name AS SubCategory, c.Name AS Category
FROM ((`tc-da-1.adwentureworks_db.product`AS p
INNER JOIN `tc-da-1.adwentureworks_db.productsubcategory` AS s
ON p.ProductSubcategoryId=s.ProductSubcategoryId)
INNER JOIN `tc-da-1.adwentureworks_db.productcategory` AS c
ON s.ProductCategoryId=c.ProductCategoryId)
ORDER BY c.Name
     """
 )
    
query_to_df(QUERY)

,ProductId,Name,ProductNumber,size,color,ProductSubcategoryId,SubCategory,Category
0,711,"Sport-100 Helmet, Blue",HL-U509-B,None,Blue,31,Helmets,Accessories
1,707,"Sport-100 Helmet, Red",HL-U509-R,None,Red,31,Helmets,Accessories
2,708,"Sport-100 Helmet, Black",HL-U509,None,Black,31,Helmets,Accessories
3,880,Hydration Pack - 70 oz.,HY-1023-70,70,Silver,32,Hydration Packs,Accessories
4,846,Taillights - Battery-Powered,LT-T990,None,None,33,Lights,Accessories
...,...,...,...,...,...,...,...,...
290,824,ML Mountain Rear Wheel,RW-M762,None,Black,17,Wheels,Components
291,825,HL Mountain Rear Wheel,RW-M928,None,Black,17,Wheels,Components
292,826,LL Road Rear Wheel,RW-R623,None,Black,17,Wheels,Components
293,894,Rear Derailleur,RD-2308,None,Silver,9,Derailleurs,Components


In [9]:
     
      QUERY = (
     """
SELECT
 p.ProductId,
 p.Name,
 p.ProductNumber,
 p.size,
 p.color,
 p.ProductSubcategoryId, 
 s.Name AS SubCategory,
 c.Name AS Category,
 p.ListPrice AS Price
FROM `tc-da-1.adwentureworks_db.product`AS p
INNER JOIN `tc-da-1.adwentureworks_db.productsubcategory` AS s
ON p.ProductSubcategoryId=s.ProductSubcategoryId
INNER JOIN `tc-da-1.adwentureworks_db.productcategory` AS c
ON s.ProductCategoryId=c.ProductCategoryId
WHERE p.ListPrice>2000 AND p.SellEndDate IS NULL
ORDER BY Price
    
     """
 )
    
query_to_df(QUERY)

,ProductId,Name,ProductNumber,size,color,ProductSubcategoryId,SubCategory,Category,Price
0,784,"Mountain-200 Black, 46",BK-M68B-46,46,Black,1,Mountain Bikes,Bikes,2294.99
1,782,"Mountain-200 Black, 38",BK-M68B-38,38,Black,1,Mountain Bikes,Bikes,2294.99
2,783,"Mountain-200 Black, 42",BK-M68B-42,42,Black,1,Mountain Bikes,Bikes,2294.99
3,779,"Mountain-200 Silver, 38",BK-M68S-38,38,Silver,1,Mountain Bikes,Bikes,2319.99
4,780,"Mountain-200 Silver, 42",BK-M68S-42,42,Silver,1,Mountain Bikes,Bikes,2319.99
5,781,"Mountain-200 Silver, 46",BK-M68S-46,46,Silver,1,Mountain Bikes,Bikes,2319.99
6,956,"Touring-1000 Yellow, 54",BK-T79Y-54,54,Yellow,3,Touring Bikes,Bikes,2384.07
7,957,"Touring-1000 Yellow, 60",BK-T79Y-60,60,Yellow,3,Touring Bikes,Bikes,2384.07
8,968,"Touring-1000 Blue, 54",BK-T79U-54,54,Blue,3,Touring Bikes,Bikes,2384.07
9,967,"Touring-1000 Blue, 50",BK-T79U-50,50,Blue,3,Touring Bikes,Bikes,2384.07


In [10]:
     
QUERY = (
     """
SELECT LocationID,
COUNT( DISTINCT WorkOrderID) AS no_of_workorder,
COUNT( DISTINCT ProductID) as no_of_product, 
SUM(ActualCost) AS total_actual_cost,
FROM `tc-da-1.adwentureworks_db.workorderrouting` 
WHERE ActualStartDate BETWEEN '2004-01-01'AND '2004-01-31'
GROUP BY  LocationID
ORDER BY total_actual_cost DESC
   
     """
 )
    
query_to_df(QUERY)

,LocationID,no_of_workorder,no_of_product,total_actual_cost
0,50,1304,48,47922.00
1,60,957,77,46893.00
2,10,409,18,37730.25
3,20,284,14,24850.00
4,40,48,3,1512.00
5,30,79,6,1145.50
6,45,31,3,1116.00


In [11]:
     
QUERY = (
     """
SELECT l.Name,
COUNT( DISTINCT w.WorkOrderID) AS no_of_workorder,
COUNT( DISTINCT w.ProductID) as no_of_product, 
SUM(w.ActualCost) AS total_actual_cost,
ROUND(AVG(DATE_DIFF(w.ActualEndDate, w.ActualStartDate, DAY)),2) AS avg_days_diff
FROM `tc-da-1.adwentureworks_db.workorderrouting` AS w
INNER JOIN `tc-da-1.adwentureworks_db.location` AS l
ON l.LocationID=w.LocationID
WHERE w.ActualStartDate BETWEEN '2004-01-01'AND '2004-01-31'
GROUP BY  l.Name
ORDER BY avg_days_diff DESC
   
     """
 )
    
query_to_df(QUERY)

,Name,no_of_workorder,no_of_product,total_actual_cost,avg_days_diff
0,Subassembly,1304,48,47922.00,11.59
1,Final Assembly,957,77,46893.00,11.46
2,Frame Forming,409,18,37730.25,11.43
3,Frame Welding,284,14,24850.00,11.31
4,Paint,48,3,1512.00,10.90
5,Debur and Polish,79,6,1145.50,10.61
6,Specialized Paint,31,3,1116.00,10.16


In [12]:
QUERY = (
     """
SELECT sales_detail.SalesOrderId
      ,sales_detail.OrderQty
      ,sales_detail.UnitPrice
      ,sales_detail.LineTotal
      ,sales_detail.ProductId
      ,sales_detail.SpecialOfferID
      ,spec_offer_product.ModifiedDate
      ,spec_offer.Category
      ,spec_offer.Description

FROM `tc-da-1.adwentureworks_db.salesorderdetail`  as sales_detail

left join `tc-da-1.adwentureworks_db.specialofferproduct` as spec_offer_product
on sales_detail.productId = spec_offer_product.ProductID

inner join `tc-da-1.adwentureworks_db.specialoffer` as spec_offer
on sales_detail.SpecialOfferID = spec_offer.SpecialOfferID

order by LineTotal desc
     """
 )
    
query_to_df(QUERY)

,SalesOrderId,OrderQty,UnitPrice,LineTotal,ProductId,SpecialOfferID,ModifiedDate,Category,Description
0,55282,26,1192.0350,27893.619000,954,4,2001-05-02 00:00:00+00:00,Reseller,Volume Discount 25 to 40
1,55282,26,1192.0350,27893.619000,954,4,2001-06-01 00:00:00+00:00,Reseller,Volume Discount 25 to 40
2,55282,26,1192.0350,27893.619000,954,4,2001-06-01 00:00:00+00:00,Reseller,Volume Discount 25 to 40
3,55282,26,1192.0350,27893.619000,954,4,2003-06-01 00:00:00+00:00,Reseller,Volume Discount 25 to 40
4,43884,14,1971.9942,27055.760424,772,2,2001-05-02 00:00:00+00:00,Reseller,Volume Discount 11 to 14
...,...,...,...,...,...,...,...,...,...
296978,53493,1,1.3740,1.374000,873,1,2001-06-01 00:00:00+00:00,No Discount,No Discount
296979,53526,1,1.3740,1.374000,873,1,2001-05-02 00:00:00+00:00,No Discount,No Discount
296980,53526,1,1.3740,1.374000,873,1,2001-06-01 00:00:00+00:00,No Discount,No Discount
296981,58981,1,1.3740,1.374000,873,1,2001-05-02 00:00:00+00:00,No Discount,No Discount


In [13]:
     
QUERY = (
     """
SELECT  WorkOrderID,
SUM(ActualCost) AS actual_cost,
FROM `tc-da-1.adwentureworks_db.workorderrouting`
WHERE ActualStartDate BETWEEN '2004-01-01'AND '2004-01-31'
GROUP BY WorkOrderID
HAVING SUM(ActualCost)>300
ORDER BY actual_cost DESC
   
     """
 )
    
query_to_df(QUERY)

,WorkOrderID,actual_cost
0,54924,316.0
1,55688,316.0
2,53449,316.0
3,54269,316.0
4,55052,316.0
...,...,...
74,54842,311.5
75,52527,311.5
76,52502,311.5
77,53310,311.5


In [14]:
     
QUERY = (
     """
SELECT  vendor.VendorId as Id,
vendor_contact.ContactId,
vendor_contact.ContactTypeId,
vendor.Name,
vendor.CreditRating,
vendor.ActiveFlag,
vendor_address.AddressId,
address.City

#mispelled vendor as Vendor
FROM `tc-da-1.adwentureworks_db.vendor` AS vendor

#named table a but referred to it as vendor
LEFT JOIN `tc-da-1.adwentureworks_db.vendorcontact` AS vendor_contact 
ON vendor.VendorId = vendor_contact.VendorId

#named table c but referred to it as vendor address
LEFT JOIN `tc-da-1.adwentureworks_db.vendoraddress` AS vendor_address
ON vendor.VendorId = vendor_address.VendorId

#named table address but referred to it as d
#join on vendorid instead of AddressID
LEFT JOIN `tc-da-1.adwentureworks_db.address` AS address 
ON vendor_address.AddressID = address.AddressID
   
     """
 )
    
query_to_df(QUERY)

,Id,ContactId,ContactTypeId,Name,CreditRating,ActiveFlag,AddressId,City
0,95,693,18,Team Athletic Co.,1,1,394,Spokane
1,101,689,18,Touring Equipment Center,1,1,395,St. Louis
2,62,612,17,Training Systems,1,1,396,Burien
3,99,634,18,Marsh,1,1,371,Woodburn
4,100,690,2,Holiday Skate & Cycle,1,1,349,Lemon Grove
...,...,...,...,...,...,...,...,...
151,71,624,18,Vista Road Bikes,3,1,401,Salem
152,36,617,18,Recreation Place,4,1,385,Salem
153,84,694,19,"Proseware, Inc.",4,0,307,Lebanon
154,63,630,17,Merit Bikes,5,1,372,Bremerton
